# Imports

In [1]:
!python -m spacy download en_core_web_sm


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "/Users/sm/Documents/GitHub/agentic_text_analyzer/.venv/lib/python3.12/site-packages/spacy/__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "/Users/sm/Documents/GitHub/agentic_text_analyzer/.venv/lib/python3.12/site-packages/spacy/errors.py", line 3, in <module>
    from .compat import Literal
  File "/User

In [2]:
# NLP packages
from nltk.corpus import reuters
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
import spacy
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
from datasets import load_dataset
import evaluate
# General packages
import os
import numpy as np
import pandas as pd
from collections import Counter, defaultdict


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/sm/Documents/GitHub/agentic_text_analyzer/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/sm/Documents/GitHub/agentic_text_analyzer/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/sm/Documents/GitHub/agentic_text_analyzer/.venv/lib/python3.12/site-p

In [3]:
# Download required NLTK data
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt')
nltk.download("punkt_tab")
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize preprocessing tools
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

print("Preprocessing tools initialized!")


Preprocessing tools initialized!


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sm/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/sm/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/sm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/sm/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/sm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/sm/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Downloading Reuters corpus

In [ ]:
def load_reuters_to_dataframe():
    """
    Load all Reuters documents into a pandas DataFrame
    """
    # Get all file IDs
    file_ids = reuters.fileids()
    
    # Initialize lists to store data
    documents = []
    doc_ids = []
    categories_list = []
    
    # Extract data from each document
    for file_id in file_ids:
        try:
            # Get document text
            doc_text = reuters.raw(file_id)
            
            # Get categories for this document
            doc_categories = reuters.categories(file_id)
            
            # Store data
            documents.append(doc_text)
            doc_ids.append(file_id)
            categories_list.append(doc_categories)
            
        except Exception as e:
            print(f"Error processing {file_id}: {e}")
            continue
    
    # Create DataFrame
    df = pd.DataFrame({
        'doc_id': doc_ids,
        'text': documents,
        'categories': categories_list
    })
    
    return df

# Load the data
print("Loading Reuters corpus...")
reuters_df = load_reuters_to_dataframe()
print(f"Loaded {len(reuters_df)} documents")
print(f"DataFrame shape: {reuters_df.shape}")


Loading Reuters corpus...
Loaded 10788 documents
DataFrame shape: (10788, 3)


# Text cleaning

In [5]:
# Text preprocessing functions
def clean_text(text):
    """
    Clean text by removing special characters, extra whitespace, etc.
    """
    if not isinstance(text, str):
        return ""
    
    # Remove HTML tags if any
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    
    # Remove extra whitespace and newlines
    text = re.sub(r'\s+', ' ', text)
    
    # Remove leading/trailing whitespace
    text = text.strip()
    # lower casing
    text = text.lower()
    
    return text


sample_string = "http. here    we go again!   "

clean_text(sample_string)

'here we go again!'

In [6]:
reuters_df['cleaned_text'] = reuters_df['text'].apply(
    lambda x: clean_text(x)
)

reuters_df.head()

,doc_id,text,categories,cleaned_text
0,test/14826,ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...,[trade],asian exporters fear damage from u.s.-japan ri...
1,test/14828,CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...,[grain],china daily says vermin eat 7-12 pct grain sto...
2,test/14829,JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...,"[crude, nat-gas]",japan to revise long-term energy demand downwa...
3,test/14832,THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...,"[corn, grain, rice, rubber, sugar, tin, trade]",thai trade deficit widens in first quarter tha...
4,test/14833,INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...,"[palm-oil, veg-oil]",indonesia sees cpo price rising sharply indone...


In [7]:
reuters_df.iloc[0,1]

'ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT\n  Mounting trade friction between the\n  U.S. And Japan has raised fears among many of Asia\'s exporting\n  nations that the row could inflict far-reaching economic\n  damage, businessmen and officials said.\n      They told Reuter correspondents in Asian capitals a U.S.\n  Move against Japan might boost protectionist sentiment in the\n  U.S. And lead to curbs on American imports of their products.\n      But some exporters said that while the conflict would hurt\n  them in the long-run, in the short-term Tokyo\'s loss might be\n  their gain.\n      The U.S. Has said it will impose 300 mln dlrs of tariffs on\n  imports of Japanese electronics goods on April 17, in\n  retaliation for Japan\'s alleged failure to stick to a pact not\n  to sell semiconductors on world markets at below cost.\n      Unofficial Japanese estimates put the impact of the tariffs\n  at 10 billion dlrs and spokesmen for major electronics firms\n  said they would v

In [8]:
reuters_df.iloc[0,3]

'asian exporters fear damage from u.s.-japan rift mounting trade friction between the u.s. and japan has raised fears among many of asia\'s exporting nations that the row could inflict far-reaching economic damage, businessmen and officials said. they told reuter correspondents in asian capitals a u.s. move against japan might boost protectionist sentiment in the u.s. and lead to curbs on american imports of their products. but some exporters said that while the conflict would hurt them in the long-run, in the short-term tokyo\'s loss might be their gain. the u.s. has said it will impose 300 mln dlrs of tariffs on imports of japanese electronics goods on april 17, in retaliation for japan\'s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost. unofficial japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports of products hit by the new taxes. "we wouldn\

# rule-based entity extraction (inefficient way using part-of-sentence tags)

In [9]:

pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x: str, flag: str):
    """
    Return list of words from sentence `x` that match POS family `flag`.
    """
    result = []
    try:
        tokens = word_tokenize(x)
        tagged = pos_tag(tokens)
        for word, tag in tagged:
            if tag in pos_family.get(flag, []):
                result.append(word)
    except Exception as e:
        print(f"Error: {e}")
    return result

In [10]:
reuters_df['noun_list'] = reuters_df['cleaned_text'].apply(lambda x: check_pos_tag(x, 'noun'))
reuters_df['adj_list'] = reuters_df['cleaned_text'].apply(lambda x: check_pos_tag(x, 'adj'))

reuters_df.head()

,doc_id,text,categories,cleaned_text,noun_list,adj_list
0,test/14826,ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...,[trade],asian exporters fear damage from u.s.-japan ri...,"[exporters, damage, rift, trade, friction, u.s...","[asian, u.s.-japan, many, far-reaching, econom..."
1,test/14828,CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...,[grain],china daily says vermin eat 7-12 pct grain sto...,"[china, eat, pct, grain, stocks, survey, provi...","[vermin, 7-12, vermin, china, inadequate, bad,..."
2,test/14829,JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...,"[crude, nat-gas]",japan to revise long-term energy demand downwa...,"[japan, energy, demand, ministry, trade, indus...","[long-term, international, long-term, japanese..."
3,test/14832,THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...,"[corn, grain, rice, rubber, sugar, tin, trade]",thai trade deficit widens in first quarter tha...,"[thai, trade, deficit, quarter, thailand, trad...","[first, first, improved, pct, raw, semi-finish..."
4,test/14833,INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...,"[palm-oil, veg-oil]",indonesia sees cpo price rising sharply indone...,"[indonesia, sees, price, oil, cpo, prices, dlr...","[cpo, indonesia, crude, palm, better, european..."


# A little more advanced extraction (NER)

In [ ]:

# load model
nlp = spacy.load("en_core_web_sm")

# Most important labels
IMPORTANT_LABELS = [
    "PERSON","ORG","GPE","LOC","DATE","TIME","MONEY","NORP","EVENT","PRODUCT"
]

def extract_important_entities(text: str, labels=None, top_k=None):
    """
    Extract named entities and return a structured result with simple importance scoring.
    - labels: filter to these labels (defaults to IMPORTANT_LABELS)
    - top_k: if set, return only the top_k entities by importance
    Importance score = frequency + 0.25*token_length (simple heuristic).
    """
    if labels is None:
        labels = IMPORTANT_LABELS

    doc = nlp(text)

    # collect entities
    ents = [(ent.text.strip(), ent.label_) for ent in doc.ents if ent.label_ in labels]

    freq = Counter([e[0] for e in ents])
    label_map = defaultdict(lambda: None)
    length_map = {}

    for ent_text, ent_label in ents:
        label_map[ent_text] = ent_label 
        length_map[ent_text] = len(ent_text.split())

    # score: freq + 0.25 * length (nicer names bubble up)
    scored = []
    for ent_text, f in freq.items():
        score = f + 0.25 * length_map[ent_text]
        scored.append({
            "text": ent_text,
            "label": label_map[ent_text],
            "frequency": f,
            "length_tokens": length_map[ent_text],
            "importance": round(score, 3),
        })

    # sort by importance desc, then frequency desc, then text
    scored.sort(key=lambda x: (-x["importance"], -x["frequency"], x["text"].lower()))

    if top_k is not None:
        scored = scored[:top_k]

    # also group by label for convenience
    by_label = defaultdict(list)
    for item in scored:
        by_label[item["label"]].append(item["text"])

    return {
        "entities_ranked": scored,   # list of dicts with scores
        "entities_by_label": dict(by_label)  # quick lookup
    }

# example
text = """
Apple announced a $2 billion investment in Austin, Texas on September 21, 2025.
Tim Cook met with officials from the U.S. Department of Commerce.
The iPhone 16 Pro launch is expected in October.
"""

res = extract_important_entities(text, top_k=10)
for e in res["entities_ranked"]:
    print(e)
print("\nBy label:", res["entities_by_label"])


{'text': 'the U.S. Department of Commerce', 'label': 'ORG', 'frequency': 1, 'length_tokens': 5, 'importance': 2.25}
{'text': 'September 21, 2025', 'label': 'DATE', 'frequency': 1, 'length_tokens': 3, 'importance': 1.75}
{'text': '$2 billion', 'label': 'MONEY', 'frequency': 1, 'length_tokens': 2, 'importance': 1.5}
{'text': 'Tim Cook', 'label': 'PERSON', 'frequency': 1, 'length_tokens': 2, 'importance': 1.5}
{'text': 'Apple', 'label': 'ORG', 'frequency': 1, 'length_tokens': 1, 'importance': 1.25}
{'text': 'Austin', 'label': 'GPE', 'frequency': 1, 'length_tokens': 1, 'importance': 1.25}
{'text': 'October', 'label': 'DATE', 'frequency': 1, 'length_tokens': 1, 'importance': 1.25}
{'text': 'Texas', 'label': 'GPE', 'frequency': 1, 'length_tokens': 1, 'importance': 1.25}

By label: {'ORG': ['the U.S. Department of Commerce', 'Apple'], 'DATE': ['September 21, 2025', 'October'], 'MONEY': ['$2 billion'], 'PERSON': ['Tim Cook'], 'GPE': ['Austin', 'Texas']}


In [12]:
def add_ner_columns(df: pd.DataFrame, text_col: str, top_k=10):
    def _extract(row_text):
        out = extract_important_entities(row_text, top_k=top_k)
        return {
            "entities_ranked": out["entities_ranked"],
            "entities_by_label": out["entities_by_label"]
        }
    tmp = df[text_col].apply(_extract)
    df["entities_ranked"] = tmp.apply(lambda d: d["entities_ranked"])
    df["entities_by_label"] = tmp.apply(lambda d: d["entities_by_label"])
    return df

reuters_df = add_ner_columns(reuters_df, "cleaned_text", 10)

reuters_df.head()

,doc_id,text,categories,cleaned_text,noun_list,adj_list,entities_ranked,entities_by_label
0,test/14826,ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...,[trade],asian exporters fear damage from u.s.-japan ri...,"[exporters, damage, rift, trade, friction, u.s...","[asian, u.s.-japan, many, far-reaching, econom...","[{'text': 'u.s.', 'label': 'GPE', 'frequency':...","{'GPE': ['u.s.', 'japan', 'taiwan', 'hong kong..."
1,test/14828,CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...,[grain],china daily says vermin eat 7-12 pct grain sto...,"[china, eat, pct, grain, stocks, survey, provi...","[vermin, 7-12, vermin, china, inadequate, bad,...","[{'text': 'china', 'label': 'GPE', 'frequency'...","{'GPE': ['china'], 'ORG': ['the china daily', ..."
2,test/14829,JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...,"[crude, nat-gas]",japan to revise long-term energy demand downwa...,"[japan, energy, demand, ministry, trade, indus...","[long-term, international, long-term, japanese...","[{'text': 'miti', 'label': 'ORG', 'frequency':...","{'ORG': ['miti', 'the ministry of internationa..."
3,test/14832,THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...,"[corn, grain, rice, rubber, sugar, tin, trade]",thai trade deficit widens in first quarter tha...,"[thai, trade, deficit, quarter, thailand, trad...","[first, first, improved, pct, raw, semi-finish...","[{'text': 'first quarter', 'label': 'DATE', 'f...","{'DATE': ['first quarter', 'the first quarter ..."
4,test/14833,INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...,"[palm-oil, veg-oil]",indonesia sees cpo price rising sharply indone...,"[indonesia, sees, price, oil, cpo, prices, dlr...","[cpo, indonesia, crude, palm, better, european...","[{'text': 'indonesia', 'label': 'GPE', 'freque...","{'GPE': ['indonesia', 'malaysia'], 'PERSON': [..."


In [13]:
reuters_df.iloc[0,3]

'asian exporters fear damage from u.s.-japan rift mounting trade friction between the u.s. and japan has raised fears among many of asia\'s exporting nations that the row could inflict far-reaching economic damage, businessmen and officials said. they told reuter correspondents in asian capitals a u.s. move against japan might boost protectionist sentiment in the u.s. and lead to curbs on american imports of their products. but some exporters said that while the conflict would hurt them in the long-run, in the short-term tokyo\'s loss might be their gain. the u.s. has said it will impose 300 mln dlrs of tariffs on imports of japanese electronics goods on april 17, in retaliation for japan\'s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost. unofficial japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports of products hit by the new taxes. "we wouldn\

In [14]:
reuters_df.iloc[0,4][:5]

['exporters', 'damage', 'rift', 'trade', 'friction']

In [15]:
reuters_df.iloc[0,5][:5]

['asian', 'u.s.-japan', 'many', 'far-reaching', 'economic']

In [16]:
reuters_df.iloc[0,6][:5]

[{'text': 'u.s.',
  'label': 'GPE',
  'frequency': 17,
  'length_tokens': 1,
  'importance': 17.25},
 {'text': 'japan',
  'label': 'GPE',
  'frequency': 12,
  'length_tokens': 1,
  'importance': 12.25},
 {'text': 'japanese',
  'label': 'NORP',
  'frequency': 4,
  'length_tokens': 1,
  'importance': 4.25},
 {'text': 'last year',
  'label': 'DATE',
  'frequency': 3,
  'length_tokens': 2,
  'importance': 3.5},
 {'text': 'taiwan',
  'label': 'GPE',
  'frequency': 3,
  'length_tokens': 1,
  'importance': 3.25}]

In [17]:
reuters_df.iloc[0,7]

{'GPE': ['u.s.', 'japan', 'taiwan', 'hong kong', 'south korea'],
 'NORP': ['japanese', 'asian'],
 'DATE': ['last year', 'the first half of the financial year'],
 'ORG': ['matsushita electric industrial co ltd &lt;mc.t>']}

# Transformer-based few-shot extraction (Experimental)

In [18]:

model_id = "google/flan-t5-base"  # light llm
tok = AutoTokenizer.from_pretrained(model_id)
mdl = AutoModelForSeq2SeqLM.from_pretrained(model_id)

def generate(prompt, max_new_tokens=256):
    x = tok(prompt, return_tensors="pt")
    y = mdl.generate(**x, max_new_tokens=max_new_tokens)
    return tok.decode(y[0], skip_special_tokens=True)

few_shot_prompt = """You extract entities and return strict JSON with keys:
{"person": str|null, "organization": str|null, "date": str|null, "city": str|null, "amount": str|null}

### Example 1
Text: "Google will invest $500 million in Dublin on 3 July 2024, Sundar Pichai said."
JSON: {"person": "Sundar Pichai", "organization": "Google", "date": "3 July 2024", "city": "Dublin", "amount": "$500 million"}

### Example 2
Text: "On Sept 21, 2025, Apple opened a new office in Austin."
JSON: {"person": null, "organization": "Apple", "date": "Sept 21, 2025", "city": "Austin", "amount": null}

### Task
Text: "Tim Cook confirmed Apple’s $2 billion project in Austin, Texas on September 21, 2025."
JSON:
"""

print(generate(few_shot_prompt))

"person": "Tim Cook", "organization": "Apple", "date": "Sept 21, 2025", "city": "Austin", "amount": "$2 billion" ###


# tf-idf-based summarization

In [19]:

def extractive_summary(text, top_n=2):
    # split into sentences
    sentences = sent_tokenize(text)

    # compute tf-idf on sentences
    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # sentence scores = sum of tf-idf weights
    scores = tfidf_matrix.sum(axis=1).A.ravel()

    # pick top_n sentences
    top_idx = np.argsort(scores)[-top_n:]
    summary = [sentences[i] for i in sorted(top_idx)]
    return " ".join(summary)

doc = """Apple announced a $2 billion investment in Austin, Texas. 
Tim Cook met with officials from the U.S. Department of Commerce. 
The project will expand manufacturing and create 2,000 jobs. 
Apple is competing with Samsung and Google in AI chips."""
print(extractive_summary(doc, top_n=2))


Tim Cook met with officials from the U.S. Department of Commerce. The project will expand manufacturing and create 2,000 jobs.


# abstractive summarization

In [20]:

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

text = """Apple announced a $2 billion investment in Austin, Texas on September 21, 2025. 
Tim Cook met with officials to discuss workforce programs and incentives. 
The project will expand local manufacturing and create 2,000 jobs."""

summary = summarizer(text, max_length=50, min_length=20, do_sample=False)[0]['summary_text']
print(summary)


Device set to use mps:0
Your max_length is set to 50, but your input_length is only 48. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)
/Users/sm/Documents/GitHub/agentic_text_analyzer/.venv/lib/python3.12/site-packages/transformers/pytorch_utils.py:339: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


Apple announced a $2 billion investment in Austin, Texas on September 21, 2025. The project will expand local manufacturing and create 2,000 jobs.


# Evals on Summarization Benchmark

In [21]:
# Eval dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test[:10]")  # sample 10
# metric
rouge = evaluate.load("rouge")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:

preds = [extractive_summary(x["article"], top_n=2) for x in dataset]
refs  = [x["highlights"] for x in dataset]

results = rouge.compute(predictions=preds, references=refs)
print(results)

{'rouge1': np.float64(0.2300424167041934), 'rouge2': np.float64(0.05107991780487174), 'rougeL': np.float64(0.15006118398979001), 'rougeLsum': np.float64(0.18343688548824633)}


In [23]:

preds = [summarizer(x["article"], max_length=50, min_length=20, do_sample=False)[0]['summary_text'] for x in dataset]
refs  = [x["highlights"] for x in dataset]

results = rouge.compute(predictions=preds, references=refs)
print(results)


{'rouge1': np.float64(0.4190002940814237), 'rouge2': np.float64(0.2154233119778794), 'rougeL': np.float64(0.3251416255254694), 'rougeLsum': np.float64(0.3659612867530163)}


## Much better results!